In [1]:
from pyquil import Program
from pyquil.gates import *
import pyquil.api as api
from pyquil.paulis import *
import numpy as np
from numpy import linalg as LA
import math
import random
import itertools

In [2]:
from grove.pyvqe.vqe import VQE
from scipy.optimize import minimize

# Setting up connection to qvm and VQE
vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'nelder-mead'})
qvm = api.QVMConnection()

ModuleNotFoundError: No module named 'grove'

In [2]:
model4site = [
    {"U": 1, "neighbors": [1,3], "hoppings": [0,1,0,1]},
    {"U": 2, "neighbors": [0,2], "hoppings": [1,0,1,0]},
    {"U": 2, "neighbors": [1,3], "hoppings": [0,1,0,1]},
    {"U": 2, "neighbors": [0,2], "hoppings": [1,0,1,0]}
]

model4site1d = [
    {"U": 1, "neighbors": [1],   "hoppings": [0,1,0,0]},
    {"U": 2, "neighbors": [0,2], "hoppings": [1,0,1,0]},
    {"U": 2, "neighbors": [1,3], "hoppings": [0,1,0,1]},
    {"U": 2, "neighbors": [2],   "hoppings": [0,0,1,0]}
]

model2site = [
    {"U": 2, "neighbors": [1], "hoppings": [0,1]},
    {"U": 2, "neighbors": [0], "hoppings": [1,0]}
]

model6site = [
    {"U": 2, "neighbors": [1,2,5], "hoppings": [0,1,1,0,0,1]},
    {"U": 2, "neighbors": [0,2,4], "hoppings": [1,0,1,0,1,0]},
    {"U": 2, "neighbors": [0,1,3], "hoppings": [1,1,0,1,0,0]},
    {"U": 2, "neighbors": [2,4,5], "hoppings": [0,0,1,0,1,1]},
    {"U": 2, "neighbors": [1,3,5], "hoppings": [0,1,0,1,0,1]},
    {"U": 2, "neighbors": [0,3,4], "hoppings": [1,0,0,1,1,0]}
]

model8site = [
    {"U": 2, "neighbors": [1,3,7], "hoppings": [0,1,0,1,0,0,0,1]},
    {"U": 2, "neighbors": [0,2,6], "hoppings": [1,0,1,0,0,0,1,0]},
    {"U": 2, "neighbors": [1,3,5], "hoppings": [0,1,0,1,0,1,0,0]},
    {"U": 2, "neighbors": [0,2,4], "hoppings": [1,0,1,0,1,0,0,0]},
    {"U": 2, "neighbors": [3,5,7], "hoppings": [0,0,0,1,0,1,0,1]},
    {"U": 2, "neighbors": [2,4,6], "hoppings": [0,0,1,0,1,0,1,0]},
    {"U": 2, "neighbors": [1,5,7], "hoppings": [0,1,0,0,0,1,0,1]},
    {"U": 2, "neighbors": [0,4,6], "hoppings": [1,0,0,0,1,0,1,0]}
]

t = t2 = t3 = 1
modelGutzwiller = [
    {"U": 2, "neighbors": [1,3,5], "hoppings": [0,t,0,t,0,t2,0,0]},
    {"U": 2, "neighbors": [0,2,6], "hoppings": [t,0,t,0,0,0,t2,0]},
    {"U": 2, "neighbors": [1,3,7], "hoppings": [0,t,0,t,0,0,0,t2]},
    {"U": 2, "neighbors": [0,2,4], "hoppings": [t,0,t,0,t2,0,0,0]},
    {"U": 0, "neighbors": [3,5,7], "hoppings": [0,0,0,t2,0,t3,0,t3]},
    {"U": 0, "neighbors": [0,4,6], "hoppings": [t2,0,0,0,t3,0,t3,0]},
    {"U": 0, "neighbors": [1,5,7], "hoppings": [0,t2,0,0,0,t3,0,t3]},
    {"U": 0, "neighbors": [2,4,6], "hoppings": [0,0,t2,0,t3,0,t3,0]}
]

model = model4site
N = len(model)
print(N)

4


In [3]:
def U():
    """
    Builds the interaction term: a_p^dagger a_q^dagger a_q a_p
    """    
    hamiltonian = ZERO()
    for i in range(N):
        hamiltonian += (model[i]["U"] / 4) * (sI() - sZ(i) - sZ(i+N) + sZ(i)*sZ(i+N))
        
    return hamiltonian

def t():
    """
    Builds the hopping term: a_p^dagger a_q
    """
    def op(s1, s2):
        """
        returns a PauliSum representing a_s1^daggar a_s2
        """    
        # for all sites in between site one (s1) and site two (s2), multiply by sigma z
        z = sI()
        for i in range(s1+1, s2):
            z *= PauliTerm('Z', i)

        return sX(s1) * z * sX(s2) + sY(s1) * z * sY(s2)
        
    hops = []
    for i in range(N):
        n = model[i]["neighbors"]
        for j in range(len(n)):
            # if a hopping is not already in the array i.e. don't add (3,0) if (0,3) is there
            if (i, n[j]) in hops or (n[j], i) in hops or i == n[j]:
                pass
            else:
                hops.append((i, n[j]))
                
    hamiltonian = ZERO()
    for hop in hops:
        # add hoppings for up and down spins
        hamiltonian += (-model[hop[0]]["hoppings"][hop[1]]) * op(hop[0], hop[1])
    for hop in hops:
        hamiltonian += (-model[hop[0]]["hoppings"][hop[1]]) * op(hop[0]+N, hop[1]+N)
    return hamiltonian * (1/2)

hamU = U()
hamt = t()

hamiltonian = hamU + hamt


In [4]:
ham_str = str(hamiltonian)
print(ham_str)
print()

ham_str = ham_str.replace("*I", "[I]")
for p in ["X", "Y", "Z"]:
    for i in reversed(range(N*2)):            
        ham_str = ham_str.replace("*{}{}".format(p, i), "[{}{}]".format(p, i))
ham_str = ham_str.replace("][", " ")
print(ham_str)

(1.75+0j)*I + (-0.25+0j)*Z0 + (-0.25+0j)*Z4 + (0.25+0j)*Z0*Z4 + (-0.5+0j)*Z1 + (-0.5+0j)*Z5 + (0.5+0j)*Z1*Z5 + (-0.5+0j)*Z2 + (-0.5+0j)*Z6 + (0.5+0j)*Z2*Z6 + (-0.5+0j)*Z3 + (-0.5+0j)*Z7 + (0.5+0j)*Z3*Z7 + (-0.5+0j)*X0*X1 + (-0.5+0j)*Y0*Y1 + (-0.5+0j)*X0*Z1*Z2*X3 + (-0.5+0j)*Y0*Z1*Z2*Y3 + (-0.5+0j)*X1*X2 + (-0.5+0j)*Y1*Y2 + (-0.5+0j)*X2*X3 + (-0.5+0j)*Y2*Y3 + (-0.5+0j)*X4*X5 + (-0.5+0j)*Y4*Y5 + (-0.5+0j)*X4*Z5*Z6*X7 + (-0.5+0j)*Y4*Z5*Z6*Y7 + (-0.5+0j)*X5*X6 + (-0.5+0j)*Y5*Y6 + (-0.5+0j)*X6*X7 + (-0.5+0j)*Y6*Y7

(1.75+0j)[I] + (-0.25+0j)[Z0] + (-0.25+0j)[Z4] + (0.25+0j)[Z0 Z4] + (-0.5+0j)[Z1] + (-0.5+0j)[Z5] + (0.5+0j)[Z1 Z5] + (-0.5+0j)[Z2] + (-0.5+0j)[Z6] + (0.5+0j)[Z2 Z6] + (-0.5+0j)[Z3] + (-0.5+0j)[Z7] + (0.5+0j)[Z3 Z7] + (-0.5+0j)[X0 X1] + (-0.5+0j)[Y0 Y1] + (-0.5+0j)[X0 Z1 Z2 X3] + (-0.5+0j)[Y0 Z1 Z2 Y3] + (-0.5+0j)[X1 X2] + (-0.5+0j)[Y1 Y2] + (-0.5+0j)[X2 X3] + (-0.5+0j)[Y2 Y3] + (-0.5+0j)[X4 X5] + (-0.5+0j)[Y4 Y5] + (-0.5+0j)[X4 Z5 Z6 X7] + (-0.5+0j)[Y4 Z5 Z6 Y7] + (-0.5+0j)[X5 X

In [5]:
from qucochemistry.utils import pyquilpauli_to_qubitop
from openfermion.transforms import get_sparse_operator
import scipy as sp

# Use exact diagonalization on the qubit hamiltonian to get the ground state energy
# It is sometimes not the lowest energy in the list below because we are restricted to the half filling sector
h = get_sparse_operator(pyquilpauli_to_qubitop(hamiltonian))


In [6]:
def generate_basis_states(n):
    """
    Generates basis states for half filling of N sites
    """
    result = []
    int_result = []
    for bits in itertools.combinations(range(n * 2), n):
        # only choosing N bits from N * 2 bits, so half filling
        s = ['0'] * n * 2
        for bit in bits:
            # first n bits are up, second n are down
            # sites go 0 to N - 1 for each spin
            s[bit] = '1'
        
        if (''.join(s)[:n].count('1') == n / 2): # half filling
            int_result.append(int(''.join(s),2)) # integer representation of bit basis
            result.append(''.join(s))            # bit (string) basis
    # order ascending
    return (result[::-1], int_result[::-1])

(result, basis_states) = generate_basis_states(N)

In [7]:
 # this code converts the matrix h, which is in the full hilbert space, into half filling
ham = np.array([[ 0 for i in range(len(result))] for j in range(len(result))])
for i in range(len(result)):
    for j in range(len(result)):
        ham[i][j] = h[basis_states[i], basis_states[j]].real
        
print(ham)

[[ 4 -1  0 ...  0  0  0]
 [-1  2 -1 ...  0  0  0]
 [ 0 -1  2 ...  0  0  0]
 ...
 [ 0  0  0 ...  1 -1  0]
 [ 0  0  0 ... -1  1 -1]
 [ 0  0  0 ...  0 -1  3]]


In [8]:
[w2, _] = np.linalg.eigh(ham)
w2[0:10]

array([-2.96999279, -2.86720252, -2.34717051, -1.86052591, -1.33570126,
       -1.32135644, -0.44027865, -0.38154893, -0.19728951, -0.16652155])

In [9]:
def U_ansatz(n, param):
    """
    Creates a circuit for e^{i \theta_U H_U}
    """
    p = Program()
    
    for i in range(n):
        p.inst(RZ(model[i]["U"]*param/4, i))
        p.inst(RZ(model[i]["U"]*param/4, i+n))
        p.inst(CNOT(i, i+n))
        p.inst(RZ(-model[i]["U"]*param/4, i+n))
        p.inst(CNOT(i, i+n))
        
    return p

In [10]:
def t_ansatz(s1, s2, param):
    """
    Creates a circuit for e^{i \theta_t H_t} for horizontal and vertical hoppings
    """
    p = Program()
    
    for i in range(s1+1, s2):
        if (i == s2-1):
            p.inst(CZ(i, i+1))
        else:
            p.inst(CNOT(i, i+1))
    
    p.inst(H(s1))
    p.inst(H(s2))
    p.inst(CNOT(s1, s2))
    p.inst(RZ(param, s2))
    p.inst(CNOT(s1, s2))
    p.inst(H(s1))
    p.inst(H(s2))
    p.inst(RX(-np.pi/2, s1))
    p.inst(RX(-np.pi/2, s2))
    p.inst(CNOT(s1, s2))
    p.inst(RZ(param, s2))
    p.inst(CNOT(s1, s2))
    p.inst(RX(-np.pi/2, s1))
    p.inst(RX(-np.pi/2, s2))  
    
    for i in reversed(range(s1+1, s2)):
        if (i == s2-1):
            p.inst(CZ(i, i+1))
        else:
            p.inst(CNOT(i, i+1))
    
    return p

In [11]:
simplified_h_t =  np.array([[ 0.0 for i in range(N)] for j in range(N)])
for i in range(N):
    for j in model[i]["neighbors"]:
        simplified_h_t[i][j] = -1.0
for i in range(N):
    simplified_h_t[i][N-1-i] = -1.51
print(simplified_h_t)
w2, v2 = LA.eigh(simplified_h_t)

for i in range(N):
    print(w2[i], v2[:,i])

[[ 0.   -1.    0.   -1.51]
 [-1.    0.   -1.51  0.  ]
 [ 0.   -1.51  0.   -1.  ]
 [-1.51  0.   -1.    0.  ]]
-2.5100000000000007 [0.5 0.5 0.5 0.5]
-0.5100000000000006 [ 0.5 -0.5 -0.5  0.5]
0.5099999999999997 [-0.5 -0.5  0.5  0.5]
2.5100000000000016 [ 0.5 -0.5  0.5 -0.5]


In [12]:
Q = np.array(
[[ 0.5,  0.5,  0.5,  0.5],
 [ 0.5,  0.5, -0.5, -0.5]]
 )

#Q = np.array([[0.5,0.5,0.5,0.5],[0.5,-0.5,-0.5,0.5]])
print(Q)
# convert to hopping direction matrix
for i in range(N):
    simplified_h_t[i][N-1-i] *= -1

from openfermion.utils import slater_determinant_preparation_circuit
from pyquil.quil import DefGate
from pyquil.quilatom import Parameter, quil_sin, quil_cos

def prepare_slater_determinant(Q): 
    """
    Prepares the Slater determinant as described in https://arxiv.org/pdf/1711.05395.pdf
    
    Args:
        Q: The (N_f x N) matrix Q with orthonormal rows which describes the Slater determinant to be prepared.
    Returns:
        p: A program that applies the sequence of Givens rotations returned 
            from slater_determinant_preparation_circuit
    """
    # Defining a controlled-RY gate
    theta = Parameter('theta')
    cry = np.array([[1,0,0,0],
                   [0,1,0,0],
                   [0,0,quil_cos(theta), -quil_sin(theta)],
                   [0,0,quil_sin(theta), quil_cos(theta)]])

    cry_definition = DefGate("CRY", cry, [theta])
    CRY = cry_definition.get_constructor()
    
    # Q is a (N_f x N) matrix
    N = Q[0].size
    N_f = len(Q)
    
    givens = slater_determinant_preparation_circuit(Q)

    def givens_rotation(tups, spin):
        """
        Performs Givens rotations
        
        Args:
            tups: tuple containing Givens rotations to be performed together
            spin: 0 represents up spin, and 1 represents down spin
        Returns:
            p: A program that applies the Givens rotations in tups
        """
        p = Program()

        for tup in tups:
            # where tup is (j, k, theta, phi)
            p.inst(CNOT(tup[1]+N*spin, tup[0]+N*spin))

            # controlled-RY
            p.inst(CRY(tup[2])(tup[0]+N*spin, tup[1]+N*spin))

            p.inst(CNOT(tup[1]+N*spin, tup[0]+N*spin))
            p.inst(RZ(tup[3], tup[1]+N*spin))
        
        return p
    
    p = Program()
    p += cry_definition
    
    # Fill first N_f orbitals for each spin
    for i in range(N_f):
        p.inst(X(i))
        p.inst(X(i+N))
  
    # Perform Givens rotations for up and down spins
    for rot in givens:
        p += givens_rotation(rot, 0)
        p += givens_rotation(rot, 1)
        
    return p

print(slater_determinant_preparation_circuit(Q))
print(prepare_slater_determinant(Q))

[[ 0.5  0.5  0.5  0.5]
 [ 0.5  0.5 -0.5 -0.5]]
[((1, 2, -1.5707963267948966, 0.0),), ((0, 1, -0.7853981633974484, 0.0), (2, 3, -0.7853981633974484, 0.0))]
DEFGATE CRY(%theta):
    1, 0, 0, 0
    0, 1, 0, 0
    0, 0, COS(%theta), -1*SIN(%theta)
    0, 0, SIN(%theta), COS(%theta)

X 0
X 4
X 1
X 5
CNOT 2 1
CRY(-pi/2) 1 2
CNOT 2 1
RZ(0) 2
CNOT 6 5
CRY(-pi/2) 5 6
CNOT 6 5
RZ(0) 6
CNOT 1 0
CRY(-0.7853981633974484) 0 1
CNOT 1 0
RZ(0) 1
CNOT 3 2
CRY(-0.7853981633974484) 2 3
CNOT 3 2
RZ(0) 3
CNOT 5 4
CRY(-0.7853981633974484) 4 5
CNOT 5 4
RZ(0) 5
CNOT 7 6
CRY(-0.7853981633974484) 6 7
CNOT 7 6
RZ(0) 7



In [72]:
 def op(s1, s2):
    """
    returns a PauliSum representing a_s1^daggar a_s2
    """
    def sigma_plus(s):
        """
        returns sigma^+ operator on side s
        sigma^+ = 1/2(X + iY)
        """
        return PauliTerm('X', s, 0.5) + PauliTerm('Y', s, complex(0.0+0.5j))
    def sigma_minus(s):
        """
        returns sigma^- operator on side s
        sigma^- = 1/2(X - iY)
        """
        return PauliTerm('X', s, 0.5) - PauliTerm('Y', s, complex(0.0+0.5j))
    
    # for all sites in between site one (s1) and site two (s2), multiply by sigma z
    z = sI()
    for i in range(s1+1, s2):
        z *= PauliTerm('Z', i)
        
    return sigma_minus(s1) * z * sigma_plus(s2)

In [33]:
# building the single particle density matrix (spdm) 
spdm = np.zeros((N,N))
for i in range(N):
    for k in range(N):
        # building an operator and vqe for each i, j        
        # element (j, i) in the spdm is a_i^dagger a_j
        spdm[k, i] = vqe_inst.expectation(prepare_slater_determinant(Q), op(s1=i, s2=k), None, qvm)
        
print(np.around(spdm, decimals=2))

NameError: name 'vqe_inst' is not defined

In [ ]:
print(vqe_inst.expectation(prepare_slater_determinant(Q), hamiltonian, None, qvm))

In [73]:
hops = []
for j in range(N):
    n = model[j]["neighbors"]
    for k in range(len(n)):
        # if a hopping is not already in the array i.e. don't add (3,0) if (0,3) is there
        if (j, n[k]) in hops or (n[k], j) in hops or j == n[k]:
            pass
        else:
            hops.append((j, n[k]))
h_hops = []
v_hops = []
for hop in hops:
    if (simplified_h_t[hop[0]][hop[1]] < 0):
        h_hops.append(hop)
    else:
        v_hops.append(hop)

def var_ansatz(params):
    """
    Prepares \sum_S [U_U(\theta_U / 2) U_h(\theta_h) U_v(\theta_v) U_U(\theta_U / 2)] | \Psi_I >
    Where | \Psi_I > is the Slater determinant for U = 0
    """
    S = 1
    p = Program()
    
    p += prepare_slater_determinant(Q)
    
    for i in range(S):
        # building U_u
        p += U_ansatz(N, params[3*i])

        # building U_t, horizontal and vertical
        # up spin horizontal hops
        for hop in h_hops:
            p += t_ansatz(hop[0], hop[1], params[3*i+1])
        # down spin horizontal hops
        for hop in h_hops:
            p += t_ansatz(hop[0]+N, hop[1]+N, params[3*i+1])

        # up spin vertical hops
        for hop in v_hops:
            p += t_ansatz(hop[0], hop[1], params[3*i+2])
        # down spin vertical hops
        for hop in v_hops:
            p += t_ansatz(hop[0]+N, hop[1]+N, params[3*i+2])  
    
        # building U_u
        p += U_ansatz(N, params[3*i])
        
    return p

In [35]:
from scipy.optimize import minimize

def expectation(x):
    """
    Expectation value for parameters, x
    """
    return vqe_inst.expectation(var_ansatz(x), hamiltonian, None, qvm)

def scipy_minimize(x):
    """
    Minimizes expectation value using parameters found in greedy search as starting point
    Powell's conjugate direction method
    """
    return minimize(expectation, x, method='powell')

def greedy_noisy_search(initial_state):
    """
    Slightly perturb the values of the points, accepting whenever this results in a lower energy
    Total of 150 evaulations of the energy
    Change step size after 30 evaluations based on number of acceptances in previous trial group
    """
    params = initial_state
    min_energy = vqe_inst.expectation(var_ansatz(initial_state), hamiltonian, None, qvm)

    def random_point(dim, step):
        """
        Generates a random point on the perimeter of a circle with radius, step
        """
        coords = [random.gauss(0, 1) for i in range(dim)]
        norm = math.sqrt(sum([i**2 for i in coords]))
        coords = [(i / norm) * step for i in coords]
        return coords

    step = 0.1
    acceptances = 0

    # Five groups of 30 trials
    for i in range(5):
        acceptances = 0
        for j in range(30):
            
            # Slightly perturb the values of the points
            coords = random_point(len(initial_state), step)
            temp_params = [sum(x) for x in zip(params, coords)]
            
            # Calculate expectation value with new parameters
            temp_energy = vqe_inst.expectation(var_ansatz(temp_params), hamiltonian, None, qvm)
            
            # Greedily accept parameters that result in lower energy
            if (temp_energy < min_energy):
                min_energy = temp_energy
                params = temp_params
                acceptances += 1
        # Update step size
        step *= (acceptances / 15)

    return {'x': params, 'energy': min_energy, 'step': step}

def global_variational(initial_params=[]):
    """
    Complete a round of greedy and Powell's optimization for six randomly chosen points
    Further optimizes the best point
    """
    points = [np.random.rand(9)*0.2 for i in range(6)]
    greedy_results = [greedy_noisy_search(x) for x in points]
    print("Done with greedy search")
    results = [scipy_minimize(i['x']) for i in greedy_results]
    print("Done with Powell's")
    res = min(results, key=lambda x:x.fun)
    print(res)
    # For the chosen point, we continue optimizing until we cannot find improvement
    res_copy = res
    energy = res_copy.fun
    while (True):
        result = greedy_noisy_search(res_copy.x)
        temp_res = scipy_minimize(result['x'])
        temp_energy = temp_res.fun
        print(temp_energy)

        # Optimizing will usually find an energy ~1e-6 lower
        # Stop it eventually
        tolerance = 0.00001
        if (abs(temp_energy - energy) > tolerance):
            res_copy = temp_res
            energy = temp_energy
        else:
            break
    return res_copy

In [ ]:
res = global_variational()
print(res)

In [74]:
print(var_ansatz([np.pi, np.pi, np.pi]))

DEFGATE CRY(%theta):
    1, 0, 0, 0
    0, 1, 0, 0
    0, 0, COS(%theta), -1*SIN(%theta)
    0, 0, SIN(%theta), COS(%theta)

X 0
X 4
X 1
X 5
X 3
X 8
CNOT 2 1
CRY(-1.1502619779790222) 1 2
CNOT 2 1
RZ(0) 2
CNOT 6 5
CRY(-1.1502619779790222) 5 6
CNOT 6 5
RZ(0) 6
CNOT 1 0
CRY(-0.8860771312043457) 0 1
CNOT 1 0
RZ(0) 1
CNOT 3 2
CRY(-0.8860771312043455) 2 3
CNOT 3 2
RZ(0) 3
CNOT 5 4
CRY(-0.8860771312043457) 4 5
CNOT 5 4
RZ(0) 5
CNOT 7 6
CRY(-0.8860771312043455) 6 7
CNOT 7 6
RZ(0) 7
CNOT 2 1
CRY(-0.42053434881587415) 1 2
CNOT 2 1
RZ(0) 2
CNOT 6 5
CRY(-0.42053434881587415) 5 6
CNOT 6 5
RZ(0) 6
RZ(pi/2) 0
RZ(pi/2) 4
CNOT 0 4
RZ(-pi/2) 4
CNOT 0 4
RZ(pi/2) 1
RZ(pi/2) 5
CNOT 1 5
RZ(-pi/2) 5
CNOT 1 5
RZ(pi/2) 2
RZ(pi/2) 6
CNOT 2 6
RZ(-pi/2) 6
CNOT 2 6
RZ(pi/2) 3
RZ(pi/2) 7
CNOT 3 7
RZ(-pi/2) 7
CNOT 3 7
H 0
H 1
CNOT 0 1
RZ(pi) 1
CNOT 0 1
H 0
H 1
RX(-pi/2) 0
RX(-pi/2) 1
CNOT 0 1
RZ(pi) 1
CNOT 0 1
RX(-pi/2) 0
RX(-pi/2) 1
H 2
H 3
CNOT 2 3
RZ(pi) 3
CNOT 2 3
H 2
H 3
RX(-pi/2) 2
RX(-pi/2) 3
CNOT 2 3
RZ(pi) 